In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas
import statsmodels.api as sm

# Process data

In [192]:
path = '/content/drive/MyDrive/Year 4 (2022 - 2023)/Fall 2022/Computational Social Science/Project/New processed data/movie_cast_AND_dummy_vars.csv'
df = pd.read_csv(path)
df.head(2)

,Unnamed: 0,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,...,is_News,is_Musical,is_Drama,is_Biography,is_Sport,is_Fantasy,is_Action,is_Adventure,is_Documentary,is_Thriller
0,0,7363,tt1179904,movie,Paranormal Activity,Paranormal Activity,0,2007,NaN,86,...,False,False,False,False,False,False,False,False,False,False
1,1,4907,tt2309260,movie,The Gallows,The Gallows,0,2015,NaN,81,...,False,False,False,False,False,False,False,False,False,True


In [193]:
adj_money = '/content/drive/MyDrive/Year 4 (2022 - 2023)/Fall 2022/Computational Social Science/Project/New processed data/budget_revenue_adj.csv'
revenue_budget_adj = pd.read_csv(adj_money)
revenue_budget_adj.head()

,tconst,primaryTitle,startYear,budget,revenue,CPI ratio 2019,budget_adj,revenue_adj
0,tt1179904,Paranormal Activity,2007,230000,194183034,1.23294,283575.56000,239415489.60101
1,tt2309260,The Gallows,2015,100000,42664410,1.07856,107856.48574,46016333.28795
2,tt0876294,Zombie Diaries,2006,9300,2000000,1.26814,11793.74797,2536289.88643
3,tt0907657,Once,2007,160000,20710513,1.23294,197269.95478,25534762.26859
4,tt0374900,Napoleon Dynamite,2004,400000,46118097,1.35350,541399.21496,62420753.77834


In [194]:
adj_money_lookup = revenue_budget_adj[['tconst', 'budget_adj', 'revenue_adj']].set_index('tconst').to_dict(orient='index')

In [195]:
df['budget_adj'] = df.apply(lambda x: adj_money_lookup[x.tconst]['budget_adj'], axis=1)
df['revenue_adj'] = df.apply(lambda x: adj_money_lookup[x.tconst]['revenue_adj'], axis=1)

In [196]:
df.columns

Index(['Unnamed: 0', 'index', 'tconst', 'titleType', 'primaryTitle',
       'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes',
       'genres', 'tmdb_id', 'og_language', 'budget', 'revenue', 'ratio', 'ROI',
       'averageRating', 'numVotes', 'Unnamed: 18', 'nconst', 'genres_list',
       'is_Family', 'is_Music', 'is_Romance', 'is_War', 'is_Comedy',
       'is_Horror', 'is_Sci-Fi', 'is_History', 'is_Crime', 'is_Western',
       'is_Mystery', 'is_News', 'is_Musical', 'is_Drama', 'is_Biography',
       'is_Sport', 'is_Fantasy', 'is_Action', 'is_Adventure', 'is_Documentary',
       'is_Thriller', 'budget_adj', 'revenue_adj'],
      dtype='object')

In [197]:
cols_to_drop = ['Unnamed: 0', 'index', 'titleType', 'originalTitle', 'isAdult', 'endYear', 'runtimeMinutes', 'tmdb_id', 'og_language', 'ratio', 'ROI', 'averageRating', 'numVotes', 'Unnamed: 18', 'budget', 'revenue']
df = df.drop(columns=cols_to_drop, axis=1)
df.head(2)

,tconst,primaryTitle,startYear,genres,nconst,genres_list,is_Family,is_Music,is_Romance,is_War,...,is_Drama,is_Biography,is_Sport,is_Fantasy,is_Action,is_Adventure,is_Documentary,is_Thriller,budget_adj,revenue_adj
0,tt1179904,Paranormal Activity,2007,"Horror,Mystery","['nm2209370', 'nm2913790', 'nm2104166', 'nm291...","['Horror', 'Mystery']",False,False,False,False,...,False,False,False,False,False,False,False,False,283575.56000,239415489.60101
1,tt2309260,The Gallows,2015,"Horror,Mystery,Thriller","['nm3790547', 'nm6018502', 'nm4090362', 'nm225...","['Horror', 'Mystery', 'Thriller']",False,False,False,False,...,False,False,False,False,False,False,False,True,107856.48574,46016333.28795


In [198]:
impurity_idx_path = '/content/drive/MyDrive/Year 4 (2022 - 2023)/Fall 2022/Computational Social Science/Project/New processed data/impurity_index.csv'
impurity_idx = pd.read_csv(impurity_idx_path)
impurity_idx.head()

,tconst,HL+M,HL+O,NL+M,NL+O,a,b,i,l,w,Total PC,Impurity Index
0,tt0035423,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4.00000,4.00000,0.00000
1,tt0118589,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,0.00000,0.00000,1.00000,4.00000,0.75000
2,tt0120467,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3.00000,4.00000,0.37500
3,tt0120667,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,3.00000,4.00000,0.37500
4,tt0120679,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,1.00000,2.00000,4.00000,0.62500


In [199]:
imp_idx_lookup = dict(zip(impurity_idx.tconst, impurity_idx['Impurity Index']))
df['d_race'] = df.apply(lambda x: imp_idx_lookup[x.tconst], axis=1)

In [200]:
starpower_path = '/content/drive/MyDrive/Year 4 (2022 - 2023)/Fall 2022/Computational Social Science/Project/New processed data/starpowers.csv'
starpower = pd.read_csv(starpower_path)
print(starpower.shape)
starpower.head()

(3540, 4)


,tconst,nconst,starpowers,avg_starpowers
0,tt0035423,"['nm0000212', 'nm0413168', 'nm0000630', 'nm000...","[6.3428571428571425, 6.666666666666667, 6.4000...",6.40238
1,tt0118589,"['nm0001014', 'nm0073160', 'nm0066586', 'nm000...","[nan, nan, 6.0, nan]",6.00000
2,tt0120467,"['nm0641168', 'nm0424682', 'nm0507915', 'nm083...","[7.7, nan, nan, 5.25]",6.47500
3,tt0120667,"['nm0344435', 'nm0004821', 'nm0262635', 'nm000...","[6.05, 4.6000000000000005, 5.933333333333334, ...",5.51458
4,tt0120679,"['nm0000161', 'nm0000547', 'nm0001691', 'nm053...","[5.869230769230768, 6.013333333333333, 6.58, 7.0]",6.36564


In [201]:
starpower_lookup = dict(zip(starpower['tconst'], starpower['avg_starpowers']))
df['starpower'] = df.apply(lambda x: starpower_lookup[x.tconst], axis=1)
df['starpower'] = df['starpower'].fillna(0)

In [202]:
cast_prop_path = '/content/drive/MyDrive/Year 4 (2022 - 2023)/Fall 2022/Computational Social Science/Project/New processed data/cast_proportion.csv'
cast_prop = pd.read_csv(cast_prop_path)
print(cast_prop.shape)
cast_prop.head()

(3540, 6)


,tconst,actor,actress,actor_proportion,actress_proportion,actor_prop
0,tt0035423,3.00000,1.00000,0.75000,0.25000,higher
1,tt0118589,2.00000,2.00000,0.50000,0.50000,equal
2,tt0120467,4.00000,0.00000,1.00000,0.00000,higher
3,tt0120667,3.00000,1.00000,0.75000,0.25000,higher
4,tt0120679,2.00000,2.00000,0.50000,0.50000,equal


In [203]:
cast_prop_lookup = dict(zip(cast_prop.tconst, cast_prop.actor_proportion))
df['actor_proportion'] = df.apply(lambda x: cast_prop_lookup[x.tconst], axis=1)

In [204]:
df.head()

,tconst,primaryTitle,startYear,genres,nconst,genres_list,is_Family,is_Music,is_Romance,is_War,...,is_Fantasy,is_Action,is_Adventure,is_Documentary,is_Thriller,budget_adj,revenue_adj,d_race,starpower,actor_proportion
0,tt1179904,Paranormal Activity,2007,"Horror,Mystery","['nm2209370', 'nm2913790', 'nm2104166', 'nm291...","['Horror', 'Mystery']",False,False,False,False,...,False,False,False,False,False,283575.56000,239415489.60101,0.62500,0.00000,0.50000
1,tt2309260,The Gallows,2015,"Horror,Mystery,Thriller","['nm3790547', 'nm6018502', 'nm4090362', 'nm225...","['Horror', 'Mystery', 'Thriller']",False,False,False,False,...,False,False,False,False,True,107856.48574,46016333.28795,0.37500,0.00000,0.50000
2,tt0876294,Zombie Diaries,2006,"Horror,Sci-Fi","['nm0014714', 'nm2428131', 'nm2408572', 'nm028...","['Horror', 'Sci-Fi']",False,False,False,False,...,False,False,False,False,False,11793.74797,2536289.88643,0.50000,0.00000,0.50000
3,tt0907657,Once,2007,"Drama,Music,Romance","['nm0360598', 'nm2461627', 'nm2008013', 'nm247...","['Drama', 'Music', 'Romance']",False,True,True,False,...,False,False,False,False,False,197269.95478,25534762.26859,0.62500,0.00000,0.75000
4,tt0374900,Napoleon Dynamite,2004,Comedy,"['nm1417647', 'nm0708293', 'nm0340973', 'nm108...",['Comedy'],False,False,False,False,...,False,False,False,False,False,541399.21496,62420753.77834,0.62500,5.45000,1.00000


In [205]:
df.columns

Index(['tconst', 'primaryTitle', 'startYear', 'genres', 'nconst',
       'genres_list', 'is_Family', 'is_Music', 'is_Romance', 'is_War',
       'is_Comedy', 'is_Horror', 'is_Sci-Fi', 'is_History', 'is_Crime',
       'is_Western', 'is_Mystery', 'is_News', 'is_Musical', 'is_Drama',
       'is_Biography', 'is_Sport', 'is_Fantasy', 'is_Action', 'is_Adventure',
       'is_Documentary', 'is_Thriller', 'budget_adj', 'revenue_adj', 'd_race',
       'starpower', 'actor_proportion'],
      dtype='object')

# Test regression #1
- Dependent variable: revenue_adj
- Independent variables:
  - d_race -- impurity index for race/ethnicity
  - starpower
  - actor_proportion (proportion of actors (male) out of all principal cast)
- Control variables:
  - genres

In [206]:
df['Intercept'] = 1
df.head(2)

,tconst,primaryTitle,startYear,genres,nconst,genres_list,is_Family,is_Music,is_Romance,is_War,...,is_Action,is_Adventure,is_Documentary,is_Thriller,budget_adj,revenue_adj,d_race,starpower,actor_proportion,Intercept
0,tt1179904,Paranormal Activity,2007,"Horror,Mystery","['nm2209370', 'nm2913790', 'nm2104166', 'nm291...","['Horror', 'Mystery']",False,False,False,False,...,False,False,False,False,283575.56000,239415489.60101,0.62500,0.00000,0.50000,1
1,tt2309260,The Gallows,2015,"Horror,Mystery,Thriller","['nm3790547', 'nm6018502', 'nm4090362', 'nm225...","['Horror', 'Mystery', 'Thriller']",False,False,False,False,...,False,False,False,True,107856.48574,46016333.28795,0.37500,0.00000,0.50000,1


In [207]:
bool_cols = ['is_Family', 'is_Music', 'is_Romance', 'is_War',
       'is_Comedy', 'is_Horror', 'is_Sci-Fi', 'is_History', 'is_Crime',
       'is_Western', 'is_Mystery', 'is_News', 'is_Musical', 'is_Drama',
       'is_Biography', 'is_Sport', 'is_Fantasy', 'is_Action', 'is_Adventure',
       'is_Documentary', 'is_Thriller']
len(bool_cols)

21

In [208]:
for col in bool_cols:
  df[col] = df[col].astype(int)

In [209]:
df.head(2)

,tconst,primaryTitle,startYear,genres,nconst,genres_list,is_Family,is_Music,is_Romance,is_War,...,is_Action,is_Adventure,is_Documentary,is_Thriller,budget_adj,revenue_adj,d_race,starpower,actor_proportion,Intercept
0,tt1179904,Paranormal Activity,2007,"Horror,Mystery","['nm2209370', 'nm2913790', 'nm2104166', 'nm291...","['Horror', 'Mystery']",0,0,0,0,...,0,0,0,0,283575.56000,239415489.60101,0.62500,0.00000,0.50000,1
1,tt2309260,The Gallows,2015,"Horror,Mystery,Thriller","['nm3790547', 'nm6018502', 'nm4090362', 'nm225...","['Horror', 'Mystery', 'Thriller']",0,0,0,0,...,0,0,0,1,107856.48574,46016333.28795,0.37500,0.00000,0.50000,1


In [210]:
y = df[['revenue_adj']]

In [211]:
cols_to_drop_X = ['tconst', 'primaryTitle', 'startYear', 'genres', 'nconst', 'genres_list', 'budget_adj', 'revenue_adj']
X = df.drop(columns=cols_to_drop_X, axis=1)

In [212]:
X.head()

,is_Family,is_Music,is_Romance,is_War,is_Comedy,is_Horror,is_Sci-Fi,is_History,is_Crime,is_Western,...,is_Sport,is_Fantasy,is_Action,is_Adventure,is_Documentary,is_Thriller,d_race,starpower,actor_proportion,Intercept
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0.62500,0.00000,0.50000,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0.37500,0.00000,0.50000,1
2,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0.50000,0.00000,0.50000,1
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.62500,0.00000,0.75000,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.62500,5.45000,1.00000,1


In [213]:
mod = sm.OLS(y, X)
res = mod.fit()

In [214]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            revenue_adj   R-squared:                       0.250
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     48.72
Date:                Mon, 07 Nov 2022   Prob (F-statistic):          1.93e-198
Time:                        19:27:28   Log-Likelihood:                -72550.
No. Observations:                3540   AIC:                         1.451e+05
Df Residuals:                    3515   BIC:                         1.453e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
is_Family         3.634e+07   1.51e+07  

In [215]:
coefs = pd.DataFrame(res.params, columns=['coefs']).reset_index().rename(columns={'index': 'coef names'})
coefs['std errors'] = res.bse.values
coefs['diff'] = abs(coefs['coefs']) - 2 * coefs['std errors']
coefs['significant'] = coefs['diff'] > 0
coefs['p_values'] = res.pvalues.values

pd.options.display.float_format = '{:.5f}'.format
coefs

,coef names,coefs,std errors,diff,significant,p_values
0,is_Family,36335022.94406,15108439.22173,6118144.50061,True,0.01623
1,is_Music,10784204.46870,19034882.08132,-27285559.69393,False,0.57106
2,is_Romance,7550930.84421,10225796.95966,-12900663.07512,False,0.46031
3,is_War,-31836570.19827,30019472.66251,-28202375.12674,False,0.28898
4,is_Comedy,-42420446.00396,9794102.07487,22832241.85421,True,0.00002
5,is_Horror,-57279979.62997,12354173.19008,32571633.24982,True,0.00000
6,is_Sci-Fi,97627023.37041,13172386.60451,71282250.16139,True,0.00000
7,is_History,-13964902.68379,18937634.12129,-23910365.55879,False,0.46092
8,is_Crime,-26833151.00760,9523950.02978,7785250.94804,True,0.00487
9,is_Western,-52776926.32119,41791309.19893,-30805692.07666,False,0.20672
